# 微分可能BP復号法

Copyright (c) 2022 Tadashi Wadayama  
Released under the MIT license  
https://opensource.org/licenses/mit-license.php

In [500]:
using LinearAlgebra
using Plots
gr()
using Random
Random.seed!(3)
using Flux

### 検査行列定義ファイル(alist形式)

このコードで利用する検査行列定義ファイルBCH_31_16_3_strip.alistは
下記のサイトからダウンロードして利用してください。

https://www.uni-kl.de/channel-codes/ml-simulation-results/

In [501]:
filename = "BCH_31_16_3_strip.alist"

"BCH_31_16_3_strip.alist"

### グローバル変数の設定

In [502]:
noise_std = 0.5
K = 100
bp_itr = 5

5

### Alist ファイルの読み込み

In [503]:
fp = open(filename, "r") 
dict = Dict()
rdict = Dict()

line = readline(fp)
u = split(line, " ") 
n = parse(Int64,u[1])
m = parse(Int64,u[2])
println("n = ", n) 
println("m = ", m)
line = readline(fp)
u = split(line, " ") 
cmax = parse(Int64,u[1])
rmax = parse(Int64,u[2]) 
println("cmax = ", cmax) 
println("rmax = ", rmax)
line = readline(fp)
line = readline(fp)
id = 1
for i in 1:n
    line = readline(fp) 
    u = split(line, " ") 
    for j in 1:cmax
        v = parse(Int64, u[j])
        if v != 0
            dict[(v,i)] = id
            rdict[id] = (v,i)
            id += 1         
        end
    end
end
esize = length(dict)

H = zeros(Int, m,n)
for x in keys(dict)
    H[x[1], x[2]] = 1
end

U = zeros(Int, n, esize)
for edge in dict
    j = edge[2]
    i = edge[1][2]
    U[i,j]=1
end

V = zeros(Int, m, esize)
for edge in dict
    j = edge[2]
    i = edge[1][1]
    V[i,j]=1
end

n = 31
m = 15
cmax = 7
rmax = 8


### パリティ検査関数

In [504]:
function parity_check(x)
    if sum(bmod.(H*x)) > 0
        return 1
    else
        return 0
    end
end

parity_check (generic function with 1 method)

### 符号化関数　

注意: ただし、左下三角がゼロのalist ファイルに限る！！
左下三角がゼロでないalistファイルの場合にはencoding error が出る

In [505]:
bmod(x) = mod(x, 2)
function random_codeword()
    x = rand(0:1, n)
    for i in 1:m
        k = m - i + 1
        parity = 0
        for j in k+1:n
            parity += H[k,j]*x[j]
        end
        x[k] = bmod(parity)
    end
    p = parity_check(x)
    if p == 0
        return x
    else 
        println("encoding error!")
    end
    return x
end

random_codeword (generic function with 1 method)

In [506]:
x = random_codeword()
parity_check(x)

0

### ミニバッチ生成関数

AWGN通信路, BPSK変調を仮定

In [507]:
function mini_batch(K)
    x = zeros(n,K)
    y = zeros(n,K)
    for j in 1:K
        x[:,j] = 1 .- 2*random_codeword()
        y[:,j] = x[:,j] + noise_std*randn(n)
    end
    return x,y
end

mini_batch (generic function with 1 method)

### 微分可能ビリーフプロパゲーション復号関数

In [508]:
eps = 1e-8
function BP_decoding(y, max_itr, xi)
    alpha = zeros(esize,K) 
    beta = zeros(esize,K)
    lambda = 2*y./noise_std^2
    for i in 1:max_itr
        beta = (U'*U-I)*alpha + U'*lambda
        beta = xi[:,i] .* beta    
        tmp = exp.((V'*V-I)*log.(abs.(tanh.(beta/2))))
        alpha_abs = 2*atanh.((1-eps)*hardtanh.(tmp))
        tmp = 1 .- 2*V'*bmod.(V*((-sign.(beta).+ 1)/2))
        alpha_sign = tmp .* sign.(beta)
        alpha = alpha_sign .* alpha_abs 
    end
    gamma = U*alpha + lambda
    return tanh.(gamma)
end

BP_decoding (generic function with 1 method)

### BP復号のデモ

In [509]:
x, y = mini_batch(K)
xi = ones(esize, bp_itr)
x̂ = sign.(BP_decoding(y, bp_itr, xi))
x̂ .!= x

31×100 BitMatrix:
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  

### 学習プロセス

In [510]:
opt = ADAM(1e-2) 
train_itr = 5000
loss(x,y) = norm(x-y)^2

xi_train = ones(esize, bp_itr)
ps = Flux.params(xi_train)

for i in 1:train_itr
    x,y = mini_batch(K)
    gs = gradient(ps) do
        x̂ = BP_decoding(y, bp_itr, xi_train)
        loss(x, x̂)
    end
    Flux.Optimise.update!(opt, ps, gs)
    if i % 100 == 0
        x,y = mini_batch(K)
        x̂ = BP_decoding(y, bp_itr, xi_train)
        println(loss(x, x̂))
    end
end

20.002892089330143
14.98789315303028
1.5144090035002484e-20
19.72392722838523
1.2951666821667145
1.5640835260946
8.102067414078419e-11
3.4368436265114776e-9
1.1054106490488635e-19
4.5917588941415515
0.0038051811572555427
0.03725714625163582
29.16563501500706
12.548524784436655
19.26149274934654
1.3670784331063207e-6
0.009766106718500582
2.1873236788625995e-19
1.3897597993941043e-21
1.987839517552349e-15
1.3861176925498768
8.816459696661134
1.687160825238931e-11
1.2909854692952767
5.495424142579338e-6
0.00044562767370730564
13.014128476913298
10.076219806097265
0.03429810027188829
0.013174973580797328
3.0441719427296825e-10
1.680803797541085e-10
3.9993126120582594
14.78569820420049
1.8287543146447897e-14
20.839149419587677
1.2840946834238818e-13
1.0682289237348626
5.340388497994448e-16
4.67512438065257
22.21982449829421
11.069200111421665
4.4228368677450836e-12
1.5516366096390687e-12
1.1132618773881438e-6
1.607732138820683
4.58325106444245
0.0011650634304978262
2.7434236959302556
0.0058

In [511]:
xi_train

120×5 Matrix{Float64}:
  0.775302  0.704564   0.454502   0.634632  0.566311
  0.484134  0.209084   0.565074   1.20242   1.09858
  0.496363  0.754794   1.15876    0.90239   1.13573
  0.332508  0.77927    1.05788    0.85726   0.981139
  1.301     0.909584   0.704776   1.21082   1.06497
  1.13468   0.83335    0.761393   0.752457  0.674755
  0.526114  0.795763   0.648948   0.35935   0.941041
  1.40554   0.80579   -0.749879   0.907115  0.901905
  0.49959   0.669526   0.510695   0.986032  0.618254
  0.818394  0.753262   0.56189    1.01356   1.37176
  1.08467   0.340679  -0.544574   0.526647  1.4082
  1.25041   0.878225   0.934764   1.05722   0.938945
  0.580589  0.597858   0.0736397  0.379899  0.436106
  ⋮                                         
  0.816669  0.756834   1.12883    1.22178   1.14953
  1.31114   0.922572   1.10161    1.14834   0.619611
 -0.322542  0.458404   0.616811   0.935747  0.690446
  0.684091  0.750312   0.725699   0.724959  0.891137
  0.336237  0.560998   0.615864   0.96

### 微分可能ビリーフプロパゲーション復号法のビット誤り率

In [512]:
total_syms = 0
error_syms = 0
num_loops = 1000

for i in 1:num_loops
    x, y = mini_batch(K)
    x̂ = BP_decoding(y, bp_itr, xi_train)
    total_syms += n*K
    error_syms += sum(sign.(x̂) .!= x)
end
println("total_syms = ", total_syms)
println("error_syms = ", error_syms)
println("symbols error rate = ", error_syms/total_syms)

total_syms = 3100000
error_syms = 1335
symbols error rate = 0.0004306451612903226


### オリジナルのビリーフプロパゲーション復号法のビット誤り率

In [513]:
total_syms = 0
error_syms = 0
num_loops = 1000
xi = ones(esize, bp_itr)

for i in 1:num_loops
    x, y = mini_batch(K)
    x̂ = BP_decoding(y, bp_itr, xi)
    total_syms += n*K
    error_syms += sum(sign.(x̂) .!= x)
end
println("total_syms = ", total_syms)
println("error_syms = ", error_syms)
println("symbols error rate = ", error_syms/total_syms)

total_syms = 3100000
error_syms = 2196
symbols error rate = 0.0007083870967741936
